### Import the libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import Binarizer
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

### Load the dataset

In [2]:
df = pd.df = pd.read_csv(r'data/train.csv', usecols=['Age','Fare','SibSp','Parch','Survived'])

In [3]:
df.dropna(inplace=True)

In [4]:
df.head()

,Survived,Age,SibSp,Parch,Fare
0,0,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,1,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,0,35.0,0,0,8.0500


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 714 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  714 non-null    int64  
 1   Age       714 non-null    float64
 2   SibSp     714 non-null    int64  
 3   Parch     714 non-null    int64  
 4   Fare      714 non-null    float64
dtypes: float64(2), int64(3)
memory usage: 33.5 KB


In [6]:
df['Family'] = df['SibSp'] + df['Parch']

In [7]:
df.head()

,Survived,Age,SibSp,Parch,Fare,Family
0,0,22.0,1,0,7.2500,1
1,1,38.0,1,0,71.2833,1
2,1,26.0,0,0,7.9250,0
3,1,35.0,1,0,53.1000,1
4,0,35.0,0,0,8.0500,0


In [8]:
df.drop(['SibSp', 'Parch'], axis=1, inplace=True)

In [9]:
df.head()

,Survived,Age,Fare,Family
0,0,22.0,7.2500,1
1,1,38.0,71.2833,1
2,1,26.0,7.9250,0
3,1,35.0,53.1000,1
4,0,35.0,8.0500,0


### Train Test Split

In [10]:
X = df.drop('Survived', axis=1)
y = df['Survived']

In [11]:
X.shape, y.shape

((714, 3), (714,))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Decision Tree without Binarization

In [13]:
clf = DecisionTreeClassifier()

#### Fit the Model

In [14]:
clf.fit(X_train, y_train)

DecisionTreeClassifier()

#### Prediction and Accuracy Score

In [15]:
y_pred = clf.predict(X_test)

In [16]:
print("Accuracy Score without Binarization: ", accuracy_score(y_test, y_pred))

Accuracy Score without Binarization:  0.6293706293706294


#### Cross Validation Score

In [17]:
clf = DecisionTreeClassifier()
print("Cross Validaton Score without Binarization: ", np.mean(cross_val_score(clf, X, y, cv = 10, scoring="accuracy")))

Cross Validaton Score without Binarization:  0.6456964006259781


### Decison Tree using Binarization

#### Binarization

In [18]:
transformer = ColumnTransformer(
    transformers=[
        ('Binarization', Binarizer(copy=False), ['Family'])
    ],
    remainder='passthrough',
    verbose_feature_names_out=True,
    verbose=True
)

In [19]:
X_train_transformed = transformer.fit_transform(X_train)
X_test_transformed = transformer.transform(X_test)

[ColumnTransformer] .. (1 of 2) Processing Binarization, total=   0.0s
[ColumnTransformer] ..... (2 of 2) Processing remainder, total=   0.0s


In [20]:
pd.DataFrame(X_train_transformed, columns=['Family','Age', 'Fare'])

,Family,Age,Fare
0,1.0,31.0,20.5250
1,1.0,26.0,14.4542
2,1.0,30.0,16.1000
3,0.0,33.0,7.7750
4,0.0,25.0,13.0000
...,...,...,...
566,1.0,46.0,61.1750
567,0.0,25.0,13.0000
568,0.0,41.0,134.5000
569,1.0,33.0,20.5250


* Family = 1 if value > threshold (0.0 by default) else 0 

In [21]:
X_train[(X_train['Age'] == 33.0) & (X_train['Fare'] == 7.7750)]

,Age,Fare,Family
719,33.0,7.775,0


#### Fit the model

In [22]:
clf = DecisionTreeClassifier()

In [23]:
clf.fit(X_train_transformed, y_train)

DecisionTreeClassifier()

#### Prediction and Accuracy Score

In [24]:
y_pred = clf.predict(X_test_transformed)

In [25]:
print("Accuracy Score with Binarization: ", accuracy_score(y_test, y_pred))

Accuracy Score with Binarization:  0.6083916083916084


#### Cross Validation Score

In [26]:
clf = DecisionTreeClassifier()
X_transformed = transformer.fit_transform(X)
print("Cross Validaton Score with Binarization: ", np.mean(cross_val_score(clf, X_transformed, y, cv = 10, scoring="accuracy")))

[ColumnTransformer] .. (1 of 2) Processing Binarization, total=   0.0s
[ColumnTransformer] ..... (2 of 2) Processing remainder, total=   0.0s
Cross Validaton Score with Binarization:  0.634585289514867
